<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [2]:
using System;


public interface ICardService
{
    void Deposit(decimal amount);
    bool Withdraw(decimal amount);
    string GetCardDetails();
}


public class CreditCard : ICardService
{
    public string CardNumber { get; set; }
    public string HolderName { get; set; }
    public string ExpiryDate { get; set; }
    protected decimal Balance { get; set; }
    public string BankName { get; set; }          
    public string Currency { get; set; }          

    public CreditCard(string cardNumber, string holderName, string expiryDate, decimal balance = 0, string bankName = "Default Bank", string currency = "RUB")
    {
        CardNumber = cardNumber;
        HolderName = holderName;
        ExpiryDate = expiryDate;
        Balance = balance;
        BankName = bankName;
        Currency = currency;
    }

    
    public void DepositFunds(decimal amount)
    {
        Balance += amount;
        Console.WriteLine($"Пополнено {amount:C}. Новый баланс: {Balance:C}");
    }

    public bool TransferTo(CreditCard targetCard, decimal amount)
    {
        if (Balance >= amount)
        {
            Balance -= amount;
            targetCard.DepositFunds(amount);
            Console.WriteLine($"Переведено {amount:C} на карту {targetCard.CardNumber}");
            return true;
        }
        Console.WriteLine("Недостаточно средств для перевода.");
        return false;
    }

    public virtual string GetInfo()
    {
        return $"Карта: {CardNumber}\nВладелец: {HolderName}\nБанк: {BankName}\nВалюта: {Currency}\nСрок действия: {ExpiryDate}";
    }

    public virtual string Pay(decimal amount)
    {
        if (Balance >= amount)
        {
            Balance -= amount;
            return $"Оплата на сумму {amount:C} прошла успешно!";
        }
        return $"Недостаточно средств! Баланс: {Balance:C}";
    }

    public virtual string CheckBalance() => $"Баланс карты: {Balance:C}";

    
    void ICardService.Deposit(decimal amount) => DepositFunds(amount);
    bool ICardService.Withdraw(decimal amount) => Balance >= amount ? (Balance -= amount) == 0 || true : false;
    string ICardService.GetCardDetails() => GetInfo();
}


public class GoldCreditCard : CreditCard
{
    public int BonusMiles { get; set; }
    public double CashbackRate { get; set; }     

    public GoldCreditCard(string cardNumber, string holderName, string expiryDate, decimal balance = 0, int bonusMiles = 0, double cashbackRate = 0.02)
        : base(cardNumber, holderName, expiryDate, balance)
    {
        BonusMiles = bonusMiles;
        CashbackRate = cashbackRate;
    }

    public void RedeemMiles(int miles)
    {
        if (miles <= BonusMiles)
        {
            BonusMiles -= miles;
            Console.WriteLine($"Использовано {miles} миль. Остаток: {BonusMiles}");
        }
        else
        {
            Console.WriteLine("Недостаточно бонусных миль.");
        }
    }

    public override string Pay(decimal amount)
    {
        if (Balance >= amount)
        {
            Balance -= amount;
            int earnedMiles = (int)(amount / 100);
            BonusMiles += earnedMiles;
            decimal cashback = amount * (decimal)CashbackRate;
            Balance += cashback;
            return $"Оплата {amount:C} успешна! Начислено {earnedMiles} миль и {cashback:C} кэшбэка.";
        }
        return $"Недостаточно средств! Баланс: {Balance:C}";
    }

    public override string GetInfo()
    {
        return base.GetInfo() + $"\nТип: Gold\nБонусные мили: {BonusMiles}\nКэшбэк: {CashbackRate:P}";
    }
}


public class PremiumCreditCard : CreditCard
{
    public string SupportAssistant { get; set; }
    public decimal CreditLimit { get; set; }       
    public decimal MonthlyFee { get; set; }        

    public PremiumCreditCard(string cardNumber, string holderName, string expiryDate, string supportAssistant, decimal balance = 0, decimal creditLimit = 20000, decimal monthlyFee = 500)
        : base(cardNumber, holderName, expiryDate, balance)
    {
        SupportAssistant = supportAssistant;
        CreditLimit = creditLimit;
        MonthlyFee = monthlyFee;
    }

    public void RequestCreditIncrease(decimal newLimit)
    {
        if (newLimit > CreditLimit)
        {
            CreditLimit = newLimit;
            Console.WriteLine($"Кредитный лимит увеличен до {CreditLimit:C}");
        }
        else
        {
            Console.WriteLine("Запрошенное значение меньше текущего лимита.");
        }
    }

    public override string CheckBalance()
    {
        if (Balance < 1000)
        {
            return $"Баланс карты: {Balance:C}\nВнимание: низкий баланс! Обратитесь к ассистенту поддержки: {SupportAssistant}";
        }
        return base.CheckBalance();
    }

    public override string GetInfo()
    {
        return base.GetInfo() + $"\nТип: Premium\nАссистент: {SupportAssistant}\nКредитный лимит: {CreditLimit:C}\nМесячная плата: {MonthlyFee:C}";
    }
}

public class CorporateCreditCard : CreditCard
{
    public string Company { get; set; }
    public int EmployeeCount { get; set; }        

    public CorporateCreditCard(string cardNumber, string holderName, string expiryDate, string company, decimal balance = 0, int employees = 0)
        : base(cardNumber, holderName, expiryDate, balance)
    {
        Company = company;
        EmployeeCount = employees;
    }

    public void AddEmployeeCard()
    {
        EmployeeCount++;
        Console.WriteLine($"Добавлена корпоративная карта. Всего сотрудников: {EmployeeCount}");
    }

    public override string GetInfo()
    {
        return base.GetInfo() + $"\nТип: Corporate\nКомпания: {Company}\nКоличество сотрудников: {EmployeeCount}";
    }
}


public class PaymentProcessor
{
    private readonly ICardService _cardService;

    public PaymentProcessor(ICardService cardService)
    {
        _cardService = cardService;
    }

    public void ProcessPayment(decimal amount)
    {
        if (_cardService.Withdraw(amount))
            Console.WriteLine($"Платеж {amount:C} выполнен через внедрение зависимости.");
        else
            Console.WriteLine("Платеж не выполнен — недостаточно средств.");
    }
}



        Console.WriteLine("=== Демонстрация работы с кредитными картами ===\n");

        CreditCard basicCard = new CreditCard("1234 5678 9012 3456", "Иван Иванов", "12/25", 5000);
        GoldCreditCard goldCard = new GoldCreditCard("2345 6789 0123 4567", "Петр Петров", "06/26", 10000, 150);
        PremiumCreditCard premiumCard = new PremiumCreditCard("3456 7890 1234 5678", "Мария Сидорова", "09/27", "support@premiumbank.ru", 800);
        CorporateCreditCard corporateCard = new CorporateCreditCard("4567 8901 2345 6789", "Алексей Козлов", "03/28", "ООО 'Технологии'", 20000);

       
        Console.WriteLine(basicCard.GetInfo());
        Console.WriteLine(basicCard.Pay(3000));
        basicCard.DepositFunds(2000);
        Console.WriteLine();

        Console.WriteLine(goldCard.GetInfo());
        Console.WriteLine(goldCard.Pay(2500));
        goldCard.RedeemMiles(50);
        Console.WriteLine();

        Console.WriteLine(premiumCard.GetInfo());
        premiumCard.RequestCreditIncrease(30000);
        Console.WriteLine(premiumCard.CheckBalance());
        Console.WriteLine();

        Console.WriteLine(corporateCard.GetInfo());
        corporateCard.AddEmployeeCard();
        Console.WriteLine();

        
        Console.WriteLine("=== Управление зависимостями ===");
        ICardService injectedService = goldCard;
        PaymentProcessor processor = new PaymentProcessor(injectedService);
        processor.ProcessPayment(5000);



=== Демонстрация работы с кредитными картами ===

Карта: 1234 5678 9012 3456
Владелец: Иван Иванов
Банк: Default Bank
Валюта: RUB
Срок действия: 12/25
Оплата на сумму ¤3,000.00 прошла успешно!
Пополнено ¤2,000.00. Новый баланс: ¤4,000.00

Карта: 2345 6789 0123 4567
Владелец: Петр Петров
Банк: Default Bank
Валюта: RUB
Срок действия: 06/26
Тип: Gold
Бонусные мили: 150
Кэшбэк: 2.000%
Оплата ¤2,500.00 успешна! Начислено 25 миль и ¤50.00 кэшбэка.
Использовано 50 миль. Остаток: 125

Карта: 3456 7890 1234 5678
Владелец: Мария Сидорова
Банк: Default Bank
Валюта: RUB
Срок действия: 09/27
Тип: Premium
Ассистент: support@premiumbank.ru
Кредитный лимит: ¤20,000.00
Месячная плата: ¤500.00
Кредитный лимит увеличен до ¤30,000.00
Баланс карты: ¤800.00
Внимание: низкий баланс! Обратитесь к ассистенту поддержки: support@premiumbank.ru

Карта: 4567 8901 2345 6789
Владелец: Алексей Козлов
Банк: Default Bank
Валюта: RUB
Срок действия: 03/28
Тип: Corporate
Компания: ООО 'Технологии'
Количество сотрудников: 